# IDX stocks Financial Report data fetch

In [60]:
import pandas_datareader as pdr
import pandas as pd
from os import listdir
from os.path import isfile, join
from datetime import datetime
import lxml
from lxml import html
import requests
import numpy as np
from random import randint
from time import sleep
from numpy import random

## Get Ticker Symbol

In [61]:
ticker = pd.read_csv('Ticker_Symbols.csv')
ticker.head()

,Ticker_Symbols
0,AALI
1,ABBA
2,ABDA
3,ABMM
4,ACES


## There are 3 financial reports to scrap
### Balance Sheet, Income Statement, and Cash Flow

## Create Function to fetch financial reports

In [62]:
def get_financial_report(ticker, finrep_type, suffix, save_path):
    url = 'https://finance.yahoo.com/quote/' + ticker + '.JK/'+ finrep_type +'?p=' + ticker
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'max-age=0',
        'Pragma': 'no-cache',
        'Referrer': 'https://google.com',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    sleeptime = random.uniform(2, 11) #to not get blocked by yahoo finance
    sleep(sleeptime)
 
    page = requests.get(url, headers)
    tree = html.fromstring(page.content)
    table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")
    try:
        assert len(table_rows) > 0
    except:
        return
 
    parsed_rows = []
 
    for table_row in table_rows:
        parsed_row = []
        el = table_row.xpath("./div")
        none_count = 0
        for rs in el:
            try:
                (text,) = rs.xpath('.//span/text()[1]')
                parsed_row.append(text)
            except ValueError:
                parsed_row.append(np.NaN)
                none_count += 1

        if (none_count < 4):
            parsed_rows.append(parsed_row)
 
    temp_df = pd.DataFrame(parsed_rows)
    temp_df.columns = temp_df.iloc[0] 
    temp_df.drop(temp_df.index[0], inplace=True)
    temp_df = temp_df.reset_index(drop=True)
    temp_df.to_csv(save_path+ ticker + '_' +suffix +'.csv', index=False)

In [63]:
# balance-sheet, financials, cash-flow
to_get = ['balance-sheet', 'financials', 'cash-flow']
suffix = ['balance_sheet', 'income_statement', 'cash_flow']
for i in range(len(to_get)):
    #sleeptime = random.uniform(2, 70) #to not get blocked by yahoo finance
    #sleep(sleeptime)
    get_financial_report('PNBN', to_get[i], suffix[i], 'database//')

In [64]:
#test
pnbn = pd.read_csv('database//PNBN_balance_sheet.csv')
pnbn

,Breakdown,12/31/2020,12/31/2019,12/31/2018,12/31/2017
0,Total Assets,"218,067,091,000","211,287,370,000","207,204,418,000","213,541,797,000"
1,Total Liabilities Net Minority Interest,"170,606,759,000","166,845,656,000","166,457,301,000","177,253,066,000"
2,Total Equity Gross Minority Interest,"47,460,332,000","44,441,714,000","40,747,117,000","36,288,731,000"
3,Total Capitalization,"62,618,010,000","62,298,337,000","59,815,203,000","47,792,512,000"
4,Common Stock Equity,"44,223,351,000","41,374,558,000","37,746,095,000","33,981,898,000"
5,Net Tangible Assets,"44,008,920,000","41,229,225,000","37,626,120,000","33,843,722,000"
6,Invested Capital,"62,618,010,000","62,298,337,000","59,815,203,000","47,792,512,000"
7,Tangible Book Value,"44,008,920,000","41,229,225,000","37,626,120,000","33,843,722,000"
8,Total Debt,"18,411,418,000","20,923,779,000","22,069,108,000","13,810,614,000"
9,Net Debt,"3,654,460,000","5,564,047,000","6,199,418,000",NaN
